In [ ]:
import os
import polars as pl
import numpy as np
from numpy.typing import NDArray
from math import ceil, floor
import matplotlib.pyplot as plt

In [ ]:
GROSS_WEIGHT = 'GrossWeight'
NUM_BINS = 100
LOWER_LIMIT_WEIGHT = 3500
UPPER_LIMIT_WEIGHT = 100000

In [ ]:
def calculate_gross_weights(filepath: str) -> tuple:

    df = pl.read_csv(filepath, separator=';', truncate_ragged_lines=True, skip_rows=6, ignore_errors=True)
    df = df.filter(pl.col(GROSS_WEIGHT) > 3500)

    gross_weights = np.array(df.select(GROSS_WEIGHT).to_series().to_list())

    return gross_weights

def filter_data(data: type) -> bool:
    return type(data) == np.int64 or type(data) == np.int32 or type(data) == np.float64 or type(data) == np.float32

def extract_location(filepath):
    if 'Aanestad' in filepath and ('Vest' in filepath or 'vest' in filepath):
        return 'Ånestad(vestgående)'
    if 'Aanestad' in filepath and ('Ost' in filepath or 'ost' in filepath or 'Øst' in filepath or 'øst' in filepath):
        return 'Ånestad(østgående)'
    if 'Øysand' in filepath:
        return 'Øysand'
    if 'Skibotn' in filepath:
        return 'Skibotn'
    if 'Verdal' in filepath:
        return 'Verdal'

In [ ]:
datasets = (
    '../WIM-data/Kistler_Øysand/20160808-31_Kistler Øysand_4913151-export(1).csv',
    '../WIM-data/Kistler_Øysand/20160901-30_Kistler Øysand_4913151-export(2).csv',
    '../WIM-data/Kistler_Øysand/20161001-31_Kistler Øysand_4913151-export(3)-fixed.csv',
    '../WIM-data/Kistler_Øysand/20161101-30_Kistler Øysand_4913151-export(4).csv',
    '../WIM-data/Kistler_Øysand/20161201-31_Kistler Øysand_4913151-export(5).csv',
    '../WIM-data/Kistler_Øysand/20170101-31_Kistler Øysand_4913151-export(6).csv',
    '../WIM-data/Kistler_Øysand/20170201-28_Kistler Øysand_4913151-export(7).csv',
    '../WIM-data/Kistler_Øysand/20170301-31_Kistler Øysand_4913151-export(8).csv',
    '../WIM-data/Kistler_Øysand/20170401-05_Kistler Øysand_4913151-export(9)-fixed.csv',
    '../WIM-data/Kistler_Øysand/20180316_1.3.1_Kistler Øysand_4913151-export(24).csv',
    '../WIM-data/Kistler_Øysand/20180401-30_Kistler Øysand_4796227-export(12).csv',
    '../WIM-data/Kistler_Øysand/20180501-31(21-26)_Kistler Øysand_4796227-export(13).csv',
    '../WIM-data/Kistler_Øysand/20180601-30(11-30)_Kistler Øysand_4796227-export(14).csv',
    '../WIM-data/Kistler_Øysand/20180701-31(01-11)_Kistler Øysand_4796227-export(15).csv',
    '../WIM-data/Kistler_Øysand/20180801-31(10-31)_Kistler Øysand_4796227-export(16).csv',
    '../WIM-data/Kistler_Øysand/20180901-30_Kistler Øysand 4796227-export(17).csv',
    '../WIM-data/Kistler_Skibotn/combinedFiles_E8_2018_kalibrert_4okt.csv',
    '../WIM-data/Kistler_Skibotn/combinedFiles_E8_2019.csv',
    '../WIM-data/Kistler_Skibotn/combinedFiles_E8_2020.csv',
    '../WIM-data/Kistler_Verdal/20150513-20150531_Kistler Verdal 4796227.csv',
    '../WIM-data/Kistler_Verdal/20150601-20150630_Kistler Verdal 4796227.csv',
    '../WIM-data/Kistler_Verdal/20150701-20150731_Kistler Verdal 4796227.csv',
    '../WIM-data/Kistler_Verdal/20150801-20150831_Kistler Verdal 4796227.csv',
    '../WIM-data/Kistler_Verdal/20150901-20150930_Kistler Verdal 4796227.csv',
    '../WIM-data/Kistler_Verdal/20151001-20151031_Kistler Verdal 4796227.csv',
    '../WIM-data/Kistler_Verdal/20151101-20151130_Kistler Verdal 4796227.csv',
    '../WIM-data/Kistler_Verdal/20151201-20151231_Kistler Verdal 4796227.csv',
    '../WIM-data/Kistler_Verdal/20160101-20160131_Kistler Verdal 4796227.csv',
    '../WIM-data/Kistler_Verdal/20160201-20160229_Kistler Verdal 4796227.csv',
    '../WIM-data/Kistler_Verdal/20160301-20160331_Kistler Verdal 4796227.csv',
    '../WIM-data/Kistler_Verdal/20160401-20160430_Kistler Verdal 4796227.csv',
    '../WIM-data/Kistler_Verdal/20160501-20160531_Kistler Verdal 4796227.csv',
    '../WIM-data/Kistler_Aanestad/20221014-20 Kistler_R3_ostg.csv',
    '../WIM-data/Kistler_Aanestad/20221014-20 Kistler_R3_vestg.csv',
    # '../WIM-data/Kistler_Aanestad/20231001-20240123_Aanestad_Ostgående.csv',
    # '../WIM-data/Kistler_Aanestad/20231001-20240123_Aanestad_Vestgående.csv',
    # '../WIM-data/Kistler_Aanestad/20240122-20240612_R3 vestgående.csv',
    # '../WIM-data/Kistler_Aanestad/20240123-20240612_R3 østgående.csv'
)


In [ ]:
intervals = np.linspace(LOWER_LIMIT_WEIGHT, UPPER_LIMIT_WEIGHT, NUM_BINS + 1)
interval_labels = [f'{intervals[i]:.1f}-{intervals[i+1]:.1f}' for i in range(len(intervals)-1)]
cols = ['location', 'startdate', 'enddate'] + interval_labels
rows = []

n_rows = ceil(len(datasets) / 3)
n_cols = 3

fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5 *n_rows))
fig.subplots_adjust(hspace=0.5)
axes = axes.flatten()

for axes_index, dataset in enumerate(datasets):

    try:
        df = pl.read_csv(dataset, separator=';', truncate_ragged_lines=True, skip_rows=6, ignore_errors=True)
        df = df.filter(pl.col(GROSS_WEIGHT) > 3500)
        df = df.rename({col: col.replace(' ', '') for col in df.columns})
        df = df.with_columns(
            pl.col("StartTime").cast(pl.Datetime(time_unit='ms')).alias('unix_timestamp')
        )

        startdate = df['unix_timestamp'].min().date().isoformat()
        enddate = df['unix_timestamp'].max().date().isoformat()

        location = extract_location(dataset)
        name_dataset = dataset.split('/')[-1]
        weights = np.array(list(filter(filter_data, calculate_gross_weights(dataset))))
        mean = np.mean(weights)
        median = np.median(weights)
        ax = axes[axes_index]
        ax.hist(weights, bins=100, color='blue', edgecolor='black')
        ax.set_xlabel('Bruttovekt kjøretøy (kilo)')
        ax.set_ylabel('Passeringer')
        ax.set_title(f'Dataset:\n{name_dataset}', fontsize=10)
        ax.text(0.5, -0.225, f'Snitt: {mean:.2f} | Median: {median:.2f}', ha='center', va='center', transform=ax.transAxes, fontsize=10)

        bins = [0] * NUM_BINS
        bin_width = (UPPER_LIMIT_WEIGHT - LOWER_LIMIT_WEIGHT) / NUM_BINS
        for weight in weights:
            if LOWER_LIMIT_WEIGHT <= weight <= UPPER_LIMIT_WEIGHT:
                bin_index = floor((weight - LOWER_LIMIT_WEIGHT) / bin_width)
                bins[min(bin_index, NUM_BINS - 1)] += 1
        rows.append([location, startdate, enddate] + bins)

    except:
        pass

df = pl.DataFrame(schema=cols, data=rows)
df.write_csv('../resultater/totalvekter.csv')

In [15]:
# Samlede totalvekter for hvert sted

df = pl.read_csv('../resultater/totalvekter.csv')
weight_columns = [col for col in df.columns if '-' in col]

df = df.groupby('location').agg([
    pl.col(col).sum().alias(col) for col in weight_columns
])

df.write_csv('../resultater/totalvekter_per_sted.csv')

C:\Users\ander\AppData\Local\Temp\ipykernel_24212\1598994095.py:6: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  df = df.groupby('location').agg([
